In [91]:
import numpy as np
import pandas as pd
p = 11
numbers = list(range(p))
pattern_list = [ [] for _ in range(p) ]
print(pattern_list)
inv = np.zeros(p,dtype=int)
for i in range(p):
    for j in range(p) : 
        if i*j%p==1:
            inv[j] = i
#print(inv)
def calculate(num1,operator,num2):
    if operator == "+":
        return (num1 + num2) % p
    if operator == '-':
        return (num1 - num2 + p)%p
    if operator == '*':
        return num1 * num2%p
    if operator == '/':
        return num1 * inv[num2]%p
operators = [ "+" , "-" , "*" , "/"]
for i in range(p):
    for j in range(p):
        for op in operators:
            if not( op=='/' and j == 0):
                pattern_list[ calculate(i,op,j) ].append(  (i,op,j) )
                #print(i,op,j,'=',calculate(i,op,j))

[[], [], [], [], [], [], [], [], [], [], []]


In [92]:
opertor_priority = { "+" : 0 , "-" : 0 , "*" : 1 , "/" : 1}

In [93]:
import random
class Node:
    def __init__(self,value):
        self.value = value
        self.left = None
        self.right = None

class RandomTree:
    def random_leaf(self,x):
        if isinstance( x.value , int ):
            return x
        else:
            if random.random() < 0.4 :
                return self.random_leaf(x.left)
            else:
                return self.random_leaf(x.right)
    def dfs(self,x):
        if isinstance( x.value,int ):
            return str(x.value)+" "
        else:
            left_str = self.dfs(x.left)
            right_str = self.dfs(x.right)
            if not(isinstance(x.left.value,int))  and opertor_priority[x.left.value] < opertor_priority[x.value]:
                left_str = "( " + left_str + ") "
            if not(isinstance(x.right.value,int)) and opertor_priority[x.value] >= opertor_priority[x.right.value]:
                right_str = "( " + right_str + ") "
            return left_str + x.value +" "+ right_str
    def display(self):
        print( self.dfs(self.root))

    def __init__(self,n):
        self.value = np.random.randint(p) 
        self.root = Node(self.value)
        self.n = n
        for _ in range(n):
            x = self.random_leaf(self.root)
            pattern = random.choice(pattern_list[x.value])
            x.left = Node(pattern[0])
            x.value = pattern[1]
            x.right = Node( pattern[2] )
        self.question = self.dfs(self.root)
    def shrink_once(self):
        x = self.root
        while True:
            if isinstance( x.left.value,str ) :
                x = x.left
            else:
                if isinstance( x.right.value,str ):
                    x = x.right
                else:
                    break
        value = int(calculate( x.left.value , x.value , x.right.value ))
        x.left = x.right = None
        x.value = value
    def shrink(self):
        result_string = "a" +" _ " * np.random.randint(0,10) + self.dfs(self.root)
        while self.root.left!=None:
            self.shrink_once()
            result_string +=  "_ " * np.random.randint(0,10) + " <pad> " + self.dfs(self.root)
        return result_string


In [94]:
def gen_csv(file_name,n): 
    trees = [ RandomTree(n) for i in range(1000)]
    values = [ tree.value for tree in trees ]
    questions = [tree.question for tree in trees] 
    CoT = [ tree.shrink() for tree in trees ]
    import pandas as pd
    df = pd.DataFrame({ "value":values , "question" : questions , "CoT" : CoT})
    display(df) 
    df.to_csv(file_name)
def get_random_data(n,seed):
    random.seed(seed)
    tree = RandomTree(n)
    value = tree.value
    question = tree.question
    CoT = tree.shrink()
    return value,question,CoT
#gen_csv("./data/len6.csv",6)

In [95]:
def do_one():
    import pandas as pd
    df =  pd.DataFrame({ "value":[] , "question" : [] , "CoT" : []})
    for i in range(11):
        for x,op,y in pattern_list[i]:
            new_row = pd.DataFrame({ "x" : [x] , "op":[op],"y":[y] ,"value":[i] , "question" : [str(x) + " " + op+ " " + "_ " * np.random.randint(5,10)+ str(y)] , "CoT" : [str(x) + " "+ op + " "+ str(y) + " = "+ "_ " * np.random.randint(5,10) + " "+ str(i)]})
            df = pd.concat([df,new_row],ignore_index=True)
            df.to_csv("one.csv",index_label=False)

In [96]:
def gen_csv_random(): 
    trees = [ RandomTree(random.randint(1,5)) for i in range(10000)]
    values = [ tree.value for tree in trees ]
    questions = [tree.question for tree in trees] 
    CoT = [  tree.shrink() for tree in trees ]
    import pandas as pd
    df = pd.DataFrame({ "value":values , "question" : questions , "CoT" : CoT})
    display(df) 
    df.to_csv(file_name)

In [97]:
def random_sentence(n):
    s = "isnlwsebdac"
    lst = []
    for _ in range(n):
        l = np.random.randint(5,10)
        sentence = " ".join([ s[random.randint(0,10)] for _ in range(l)])
        lst.append(sentence)
    return lst
import numpy as np
def gen_findnum():
    n = np.random.randint(12)
    nums = np.random.choice( 11,n,replace=True ).tolist()
    k = np.random.randint(0,10)
    a = list(map(str , nums)) + k * ["_"]
    random.shuffle(a)
    question = "OO_" + "_".join(a)
    value = str(n)
    CoT = question + "=" + value
    return pd.DataFrame({ "value":[value] , "question" : [question] , "CoT" : [CoT]})
def gen_copy_fixed():
    st = 80 
    #n = np.random.randint(5,35)
    n = np.random.randint(40,60)
    t = int(np.random.rand(1) * n ) 
    lst = ["0"] * t + ["1"] * (n-t) + ["_"] * (st-n)
    random.shuffle(lst)
    
    question = " ".join(lst)
    lst = [item for item in lst if item!='_']
    value = " [ " +  " ".join(lst) +" ] "
    CoT = question + " = " + value
    return pd.DataFrame({ "value":[value] , "question" : [question] , "CoT" : [CoT]})

def maxlen(df):
    return max(df["CoT"].apply(lambda s:len(s.split())))

def gen_csv_func(func,filename,n=10000):
    rows = []
    for i in range(n):
        rows.append(func())
    df = pd.concat(rows , ignore_index=True)
    display(df)
    print("maxlen is", maxlen(df))
    df.to_csv(filename)
#global sentence 
#sentence = random_sentence(50)
#gen_csv_func(gen_copy_fixed,"./data/copy_fixed_hard.csv")

In [98]:
def gen_sort():
    n = np.random.randint(3,7)
    nums = np.random.choice(11,n,replace=True).tolist()
    question =  "OI_" + "_".join(list(map(str,nums)))
    nums = sorted(nums)
    value = "_".join(list(map(str,nums)))
    CoT = question + "=" + value
    return pd.DataFrame({ "value":[value] , "question" : [question] , "CoT" : [CoT]})


In [99]:

def gen_arith_len(): 
    
    n = 6
    tree = RandomTree(n)
    values = tree.value 
    questions = "x " +  "_ " * np.random.randint(0,10) + tree.dfs(tree.root)
    result_string = "_ " * np.random.randint(1,10)  + tree.dfs(tree.root)
    while tree.root.left!=None:
        tree.shrink_once()
        result_string +=  "= " + "_ " * np.random.randint(1,10)  + tree.dfs(tree.root)
    CoT =  questions + " = " + result_string
    import pandas as pd
    df = pd.DataFrame({ "value":[values ], "question" :[questions ], "CoT" : [CoT]})
    return df

In [100]:
def gen_arith_random(): 
    #n = 6
    n = np.random.choice( 5 , p = [0.1,0.1,0.1,0.3,0.4]) + 1
    tree = RandomTree(n)
    values = tree.value 
    l = 20
    #l = np.random.randint( 1,40 )
    questions = "x " +  "_ " * (np.random.randint(0,l)-1) + tree.dfs(tree.root)
    result_string = ""
    while tree.root.left!=None:
        tree.shrink_once()
        result_string +=  " = " + "_ " * (np.random.randint(0,l)-1)  + tree.dfs(tree.root)
    CoT =  questions + result_string
    import pandas as pd
    df = pd.DataFrame({ "value":[values ], "question" :[questions ], "CoT" : [CoT]})
    return df
def random_normal_int(n,m):
    a = np.abs(np.random.randn(n))
    a /= np.sum(a)
    a *= m
    a = np.array( a,dtype = int)
    a[n-1] += m - a.sum()
    return a
def gen_findrank():
    n = np.random.randint( 3,11 )
    m = np.random.randint( 20,30 )
    nums = np.random.choice( 11,n,replace=False )
    cnt = random_normal_int(n,m)
    a = []
    for i in range(n):
        a += [i] * cnt[i]
    random.shuffle(a)
    a = np.array( a, dtype = int)
    #print(a)
    inp = np.zeros(m,dtype=int)
    out = np.zeros(m,dtype=int)
    for i in range(n):
        pos = a==i
        rank = np.cumsum(np.array(pos,dtype=int)) % 11
        inp[pos] = nums[i]
        out[pos] = rank[pos]
    inp_str = "y " + "_ " * np.random.randint(1,10)  + " ".join(list(map(str,inp.tolist())))
    out_str = " ".join(["<pad>"] + list(map(str,out.tolist())))
    df = pd.DataFrame( {"question" : [inp_str] , "value":[out_str] , "CoT" : [inp_str + " = " + out_str]} )
    return df

def gen_copy():
    st = 80 + np.random.randint(-10,10)
    n = np.random.randint( 3,11 )
    m = np.random.randint( 5,50 )
    nums = np.random.choice( 11,n,replace=False )
    cnt = random_normal_int(n,m)
    a = ["_"] * (st - m)
    for i in range(n):
        a += [str(i)] * cnt[i]
    random.shuffle(a)
    question = "c " + " ".join(a)
    a = [ c for c in a if c!="_"]
    value = "[ " + " ".join(a) + " ]"
    CoT = question + " = " + value
    return pd.DataFrame({ "value":[value] , "question" : [question] , "CoT" : [CoT]})
def gen_dis():
    n = np.random.randint( 3,11 )
    m = np.random.randint( 20,30 )
    nums = np.random.choice( 11,n,replace=False )
    cnt = random_normal_int(n,m)
    a = []
    for i in range(n):
        a += [i] * cnt[i]
    random.shuffle(a)
    a = np.array( a, dtype = int)
    inp = np.zeros(m,dtype=int)
    out = np.zeros(m,dtype=int)
    for i in range(n):
        pos = a==i
        index = np.where(pos)
        #print(index)
        dis = np.diff( np.pad( index[0],(1,0))  ) % 11
        #print(dis)
        inp[pos] = nums[i]
        out[index] = dis
    inp_str = "d " + "_ " * np.random.randint(1,10)  + " ".join(list(map(str,inp.tolist())))
    out_str = " ".join(["<pad>"] + list(map(str,out.tolist())))
    df = pd.DataFrame( {"question" : [inp_str] , "value":[out_str] , "CoT" : [inp_str + " = " + out_str]} )
    return df
def gen_rotate():
    n = np.random.randint( 4,11 )
    inp = []
    out = []
    a = np.arange(n)
    inp += ["r"] + ["_"] * np.random.randint(1,10) + [n]
    out += a.tolist() 
    m = np.random.randint( 5,10 )
    for i in range(m): 
        inp += "|"
        cnt = np.random.randint( 2,n )
        nums = np.random.choice( n,cnt,replace=False)
        inp += nums.tolist()
        nums_to = np.zeros_like(nums)
        nums_to[:cnt-1] = nums[1:]
        nums_to[cnt-1] = nums[0]
        #print(nums)
        #print(nums_to)
        a[nums] = a[nums_to]
        out += "="
        out += a.tolist()
    #print(len(inp) + len(out))
    inp = " ".join(list(map(str,inp)))
    out = " ".join(list(map(str,out)))
    df = pd.DataFrame( {"question" : [inp] , "value":[out] , "CoT" : [inp + " = " + out]} )
    return df


In [101]:
'''
gen_arith_random x
gen_findrank y
gen_copy c
gen_dis d
gen_rotate r
'''

def gen_merge():
    func_list = [ gen_arith_random,gen_findrank,gen_copy,gen_dis,gen_rotate ]
    idx = np.random.choice( 5 , p = [ 0.2,0.2,0.2,0.2,0.2 ] )
    df = func_list[idx]()
    df["type"] = "xycdr"[idx]
    return df

In [102]:
#gen_csv_func(filename = "./data/merge.csv" , func = gen_merge , n = 50000)

In [103]:
def gen_sentence(n):
    chars = "rmdchpsf"
    lst = []
    for _ in range(n):
        length = np.random.randint( 4,10 )
        s = ""
        for _ in range(length):
            s += chars[np.random.randint(8)]
        lst.append(s)
    lst = list(set(lst))
    return pd.DataFrame({"sentence" : lst})
#display(gen_sentence(400))
#gen_sentence(400).to_csv("sentence.csv",index_label = False)

In [104]:

def gen_normal(): 
    
    #n = np.random.choice( 5 , p = [0.1,0.1,0.1,0.1,0.6]) + 1
    n = 7
    tree = RandomTree(n)
    values = tree.value
    questions = tree.dfs(tree.root)
    result_string = ""
    while tree.root.left!=None:
        tree.shrink_once()
        result_string +=  " = " + tree.dfs(tree.root)
    CoT =  questions + result_string
    import pandas as pd
    df = pd.DataFrame({ "value":[values ], "question" :[questions ], "CoT" : [CoT]})
    return df

In [105]:
def gen_random_func(prob): 
        #n = 6
    def gen():
        n = np.random.choice( len(prob) , p = prob ) + 1
        tree = RandomTree(n)
        values = tree.value 
        l = 20
        #l = np.random.randint( 1,40 )
        questions =   tree.dfs(tree.root) + " <ans> "
        result_string = ""
        while tree.root.left!=None:
            tree.shrink_once()
            result_string +=  " = " +  tree.dfs(tree.root)
        CoT =  questions + result_string
        import pandas as pd
        df = pd.DataFrame({ "value":[values ], "question" :[questions ], "CoT" : [CoT]})
        return df
    return gen
def gen_len_func(n): 
        #n = 6
    def gen():
        tree = RandomTree(n)
        values = tree.value 
        l = 20
        #l = np.random.randint( 1,40 )
        questions = "<bos> " +  tree.dfs(tree.root) 
        result_string = ""
        while tree.root.left!=None:
            tree.shrink_once()
            result_string +=  " = " +  tree.dfs(tree.root)
        CoT =  questions + " <ans> " + result_string + "<eos>"
        import pandas as pd
        df = pd.DataFrame({ "value":[values ], "question" :[questions ], "CoT" : [CoT]})
        return df
    return gen


In [106]:
import os
os.makedirs("data",exist_ok=True)
#gen_csv_func(func= gen_random_func( [ 0.1,0.1,0.1,0.1,0.6]) ,filename="./data/arith.csv" , n=20000)
"""gen_csv_func(func= gen_len_func(5) ,filename="./data/arith.csv" , n=20000)
gen_csv_func(func= gen_len_func(5) ,filename="./data/len5.csv" , n=1000)
gen_csv_func(func= gen_len_func(6) ,filename="./data/len6.csv" , n=1000)
gen_csv_func(func= gen_len_func(7) ,filename="./data/len7.csv" , n=1000)
gen_csv_func(func= gen_len_func(8) ,filename="./data/len8.csv" , n=1000)"""
#gen_csv_func(func= gen_len_func(15) ,filename="./data/one-shot.csv" , n=1)

'gen_csv_func(func= gen_len_func(5) ,filename="./data/arith.csv" , n=20000)\ngen_csv_func(func= gen_len_func(5) ,filename="./data/len5.csv" , n=1000)\ngen_csv_func(func= gen_len_func(6) ,filename="./data/len6.csv" , n=1000)\ngen_csv_func(func= gen_len_func(7) ,filename="./data/len7.csv" , n=1000)\ngen_csv_func(func= gen_len_func(8) ,filename="./data/len8.csv" , n=1000)'

In [107]:
"""def gen_one_shot():
    long = [gen_len_func(15)()] 
    r = gen_random_func([0.1,0.1,0.1,0.1,0.6])
    for i in range(499):
        long += [r()]
    df = pd.concat(long)
    display(df)
    df.to_csv("data/one-shot.csv")
gen_one_shot()"""

'def gen_one_shot():\n    long = [gen_len_func(15)()] \n    r = gen_random_func([0.1,0.1,0.1,0.1,0.6])\n    for i in range(499):\n        long += [r()]\n    df = pd.concat(long)\n    display(df)\n    df.to_csv("data/one-shot.csv")\ngen_one_shot()'

In [108]:
def gen_long():
    long = [] 
    for i in range(1,11):
        long += [gen_len_func(i)() for _ in range(2000)]
    df = pd.concat(long)
    display(df)
    df.to_csv("data/train.csv")
gen_long()
for i in range(10,21):
    gen_csv_func(func= gen_len_func(i) ,filename=f"./data/len{i}.csv" , n=1000)

,value,question,CoT
0,6,<bos> 2 + 4,<bos> 2 + 4 <ans> = 6 <eos>
0,6,<bos> 5 * 10,<bos> 5 * 10 <ans> = 6 <eos>
0,5,<bos> 3 + 2,<bos> 3 + 2 <ans> = 5 <eos>
0,5,<bos> 8 + 8,<bos> 8 + 8 <ans> = 5 <eos>
0,1,<bos> 1 + 0,<bos> 1 + 0 <ans> = 1 <eos>
...,...,...,...
0,4,<bos> 0 - 7 / 3 + ( 7 + 3 - 8 - ( 2 * 8 + ( 5 ...,<bos> 0 - 7 / 3 + ( 7 + 3 - 8 - ( 2 * 8 + ( 5 ...
0,2,<bos> 9 + 10 / 10 - 8 * ( 1 * 4 ) - 5 / 5 * ( ...,<bos> 9 + 10 / 10 - 8 * ( 1 * 4 ) - 5 / 5 * ( ...
0,5,<bos> ( 10 * 8 + 4 * 3 ) / ( 1 + 9 - ( 0 - 10 ...,<bos> ( 10 * 8 + 4 * 3 ) / ( 1 + 9 - ( 0 - 10 ...
0,7,<bos> 9 - 7 + ( 8 - 10 ) - ( 6 + 10 ) * ( 10 *...,<bos> 9 - 7 + ( 8 - 10 ) - ( 6 + 10 ) * ( 10 *...


,value,question,CoT
0,1,<bos> 3 * ( 3 / 5 ) / ( 8 * 3 ) * ( ( 3 + 1 ) ...,<bos> 3 * ( 3 / 5 ) / ( 8 * 3 ) * ( ( 3 + 1 ) ...
1,4,<bos> 6 / 3 - 5 * 0 - ( 4 / ( 3 - 4 ) + ( 6 - ...,<bos> 6 / 3 - 5 * 0 - ( 4 / ( 3 - 4 ) + ( 6 - ...
2,8,<bos> ( 5 + ( 1 * 8 - 1 ) ) * ( ( 0 - ( 0 + 6 ...,<bos> ( 5 + ( 1 * 8 - 1 ) ) * ( ( 0 - ( 0 + 6 ...
3,8,<bos> ( 10 * 6 - ( 8 + 10 / 5 ) ) / ( 10 + 4 -...,<bos> ( 10 * 6 - ( 8 + 10 / 5 ) ) / ( 10 + 4 -...
4,1,<bos> ( 6 - 6 * ( 1 + 7 ) ) / ( 3 + 9 + ( 5 - ...,<bos> ( 6 - 6 * ( 1 + 7 ) ) / ( 3 + 9 + ( 5 - ...
...,...,...,...
995,2,<bos> ( 5 + ( 5 + 6 + 1 ) ) / ( 9 + ( 2 + 6 ) ...,<bos> ( 5 + ( 5 + 6 + 1 ) ) / ( 9 + ( 2 + 6 ) ...
996,7,<bos> 3 * 7 * ( 9 - ( 8 * 9 - 10 ) ) + ( 6 / 2...,<bos> 3 * 7 * ( 9 - ( 8 * 9 - 10 ) ) + ( 6 / 2...
997,6,<bos> 5 * ( 2 + 8 ) + ( 5 - 5 ) / ( ( 2 - 3 ) ...,<bos> 5 * ( 2 + 8 ) + ( 5 - 5 ) / ( ( 2 - 3 ) ...
998,7,<bos> ( 1 - 2 - 4 ) / ( 3 - ( 4 - 2 ) - ( 8 - ...,<bos> ( 1 - 2 - 4 ) / ( 3 - ( 4 - 2 ) - ( 8 - ...


maxlen is 220


,value,question,CoT
0,4,<bos> 3 + 6 * 7 - ( 6 * ( 3 + 3 ) - ( 9 + 4 + ...,<bos> 3 + 6 * 7 - ( 6 * ( 3 + 3 ) - ( 9 + 4 + ...
1,8,<bos> 9 / 8 + 9 * ( 6 + 0 ) - ( 6 - 10 + 9 * 4...,<bos> 9 / 8 + 9 * ( 6 + 0 ) - ( 6 - 10 + 9 * 4...
2,1,<bos> 10 - 4 - 10 * 3 * 2 + ( 3 * 3 + 5 + ( 8 ...,<bos> 10 - 4 - 10 * 3 * 2 + ( 3 * 3 + 5 + ( 8 ...
3,3,<bos> 3 * 3 - 3 * ( 4 + 0 ) / ( 1 * ( 4 - 5 ) ...,<bos> 3 * 3 - 3 * ( 4 + 0 ) / ( 1 * ( 4 - 5 ) ...
4,7,<bos> 6 * 10 / ( 9 / ( 6 - 2 ) * ( 10 / 8 ) ) ...,<bos> 6 * 10 / ( 9 / ( 6 - 2 ) * ( 10 / 8 ) ) ...
...,...,...,...
995,3,<bos> 7 - 7 + 7 + ( 1 / 3 - 7 ) + ( 7 - 0 + ( ...,<bos> 7 - 7 + 7 + ( 1 / 3 - 7 ) + ( 7 - 0 + ( ...
996,7,<bos> ( 4 + 0 + 1 / 9 ) * ( 9 * 3 + 10 + 5 / 8...,<bos> ( 4 + 0 + 1 / 9 ) * ( 9 * 3 + 10 + 5 / 8...
997,4,<bos> 7 / ( 8 / 2 ) / ( 6 * 9 / 4 / ( ( 3 - 9 ...,<bos> 7 / ( 8 / 2 ) / ( 6 * 9 / 4 / ( ( 3 - 9 ...
998,2,<bos> ( 9 - 8 + 5 ) / ( ( 9 + 9 ) * 6 ) - 10 *...,<bos> ( 9 - 8 + 5 ) / ( ( 9 + 9 ) * 6 ) - 10 *...


maxlen is 262


...

maxlen is 652


,value,question,CoT
0,9,<bos> 0 * 8 - ( ( 6 - 8 ) * ( 4 + 8 ) + 9 / ( ...,<bos> 0 * 8 - ( ( 6 - 8 ) * ( 4 + 8 ) + 9 / ( ...
1,10,<bos> ( 5 - 7 ) / ( 9 - 10 ) + ( 8 / 4 - 2 + 1...,<bos> ( 5 - 7 ) / ( 9 - 10 ) + ( 8 / 4 - 2 + 1...
2,3,<bos> 4 * ( 4 / 3 ) * ( ( 1 - 9 ) * 0 ) - ( 2 ...,<bos> 4 * ( 4 / 3 ) * ( ( 1 - 9 ) * 0 ) - ( 2 ...
3,8,<bos> ( 5 + 8 - 7 ) / 7 * ( ( 9 + 7 ) / ( 2 / ...,<bos> ( 5 + 8 - 7 ) / 7 * ( ( 9 + 7 ) / ( 2 / ...
4,2,<bos> 6 - 3 - ( 2 + ( 8 - 1 ) ) - ( 5 / ( 8 + ...,<bos> 6 - 3 - ( 2 + ( 8 - 1 ) ) - ( 5 / ( 8 + ...
...,...,...,...
995,2,<bos> 3 / 7 / 4 * ( 8 + ( ( 6 + 4 ) / 8 + 3 ) ...,<bos> 3 / 7 / 4 * ( 8 + ( ( 6 + 4 ) / 8 + 3 ) ...
996,6,<bos> ( ( 8 + 1 ) / ( 8 * 6 ) - 9 / ( 10 - 5 )...,<bos> ( ( 8 + 1 ) / ( 8 * 6 ) - 9 / ( 10 - 5 )...
997,3,<bos> 9 * ( 10 * 5 - 10 / ( 7 + 3 ) ) / ( 1 * ...,<bos> 9 * ( 10 * 5 - 10 / ( 7 + 3 ) ) / ( 1 * ...
998,6,<bos> 0 + 5 * 1 / 4 * 7 - ( 10 - 0 / 5 ) - 1 /...,<bos> 0 + 5 * 1 / 4 * 7 - ( 10 - 0 / 5 ) - 1 /...


maxlen is 708


,value,question,CoT
0,9,<bos> 5 / ( 3 / 7 ) - 3 / 8 / ( 6 * 4 ) + ( 8 ...,<bos> 5 / ( 3 / 7 ) - 3 / 8 / ( 6 * 4 ) + ( 8 ...
1,10,<bos> ( 8 + 4 ) / ( 7 - ( 2 - 8 ) ) * ( 9 * 9 ...,<bos> ( 8 + 4 ) / ( 7 - ( 2 - 8 ) ) * ( 9 * 9 ...
2,5,<bos> 0 / 2 - ( 4 - 2 / 3 + ( 10 + 8 ) ) - ( 6...,<bos> 0 / 2 - ( 4 - 2 / 3 + ( 10 + 8 ) ) - ( 6...
3,7,<bos> 9 / 7 * ( 3 + 5 ) + ( 6 + 9 + 2 - ( 4 - ...,<bos> 9 / 7 * ( 3 + 5 ) + ( 6 + 9 + 2 - ( 4 - ...
4,5,<bos> ( 2 / 5 - 9 / 2 * ( 0 / ( 5 + 7 ) ) ) / ...,<bos> ( 2 / 5 - 9 / 2 * ( 0 / ( 5 + 7 ) ) ) / ...
...,...,...,...
995,2,<bos> ( 8 + 5 * 6 + 9 / ( 5 + 0 ) / ( 8 + 4 ) ...,<bos> ( 8 + 5 * 6 + 9 / ( 5 + 0 ) / ( 8 + 4 ) ...
996,10,<bos> ( 9 * 10 - 1 / 3 ) / ( 10 - 9 - 10 / 2 )...,<bos> ( 9 * 10 - 1 / 3 ) / ( 10 - 9 - 10 / 2 )...
997,5,<bos> ( 8 * 9 + 9 ) * ( 5 * 2 ) + 8 * ( 6 + 10...,<bos> ( 8 * 9 + 9 ) * ( 5 * 2 ) + 8 * ( 6 + 10...
998,6,<bos> 5 * 4 + ( 8 - 1 ) / ( 6 - 5 - 8 / 4 ) - ...,<bos> 5 * 4 + ( 8 - 1 ) / ( 6 - 5 - 8 / 4 ) - ...


maxlen is 794
